# Example 6: Counting and measuring freshwater snails

In this example we will use thresholding and watershed algorithms to count freshwater snails.

<div class="row; text-align: left">
    
<div class="col-md-6">
    
![Before](_assets/ex6_before.jpg)
    
**Input** - Snails photographed from a camera stand. Variable brightness across the tray and snail clumping are the biggest challenges.
</div>
<div class="col-md-6">

![After](_assets/ex6_after.jpg)
    
**Results** - After applying adaptive thresholding and a watershed algorithm, the snail separate well from the background. Now we can count them, and measure size, shape and colouration
</div>
</div>


## Low throughput

First, we test a single image with the low throughput workflow.

In [1]:
import phenopype as pp
import os 

image_dir = "images/snails1.jpg"

ct = pp.load_image(image_dir, 
                   cont=True,  # load as container
                   dirpath= "../_temp/output/ex6",  # specify save-directory
                   save_suffix = "ex6") # give files a save-suffix (e.g. "contours_ex6.csv")

Directory to save files set at - E:\git_repos\phenopype\_temp\output\ex6
Nothing loaded.


The `load` method of a `container` attempts to load any previously saved results, masks, etc. from the specified `dirpath`. `reset` will only reset the modified images, as well as image and contour DataFrames. This will preserve any drawn masks or other data stemming from user interaction.  

In [22]:
ct.load()
ct.reset()

Nothing loaded.


Draw a mask around the snails inside the tray by dragging a rectangle around them - finish with `Enter`, abort with `Esc`.

In [20]:
pp.preprocessing.create_mask(ct)

- mask with label mask1 already created (overwrite=False)


Now we'll blur the image a little, convert it to a binary image, and look at the results:

In [26]:
pp.segmentation.blur(ct)
pp.segmentation.threshold(ct, 
                          method="adaptive", 
                          blocksize=59, ## relatively low sensitivity
                          constant=10) ## relatively high subtraction factor from the result
pp.show_image(ct) # `ct` automatically shows the last edit within the container, `ct.image` would also work

- include mask "mask1" pixels


The break up the clumping, we apply the watershed algorithm to the binarized image:

In [24]:
pp.segmentation.watershed(ct, distance_cutoff=0.5) # , iterations=1, kernel_size=1
pp.show_image(ct)

With this image we can find the contours - for watershed we *have* to select the `"ccomp"` option.

In [25]:
pp.segmentation.find_contours(ct, 
                              retrieval="ccomp", # this finds the splitted rather than the outer contours
                              min_area=200) # noise removal

Now we draw the contours...

In [13]:
pp.visualization.select_canvas(ct, # onto which image should the contours be draw
                               canvas="raw") # raw = original image
pp.visualization.draw_contours(ct, 
                               fill=0, 
                               line_width=2, 
                               watershed=True) # this flag needs to be added
pp.show_image(ct.canvas)

- raw image


... and save them, as well as the masks (if we need to redo this) and the canvas for quality control.

In [15]:
pp.export.save_canvas(ct, canvas=raw, name="raw", resize=1)
pp.export.save_masks(ct)
pp.export.save_contours(ct)

- canvas saved under ../_temp/output/ex6\canvas_ex6.jpg (overwritten).
- masks saved under ../_temp/output/ex6\masks_ex6.csv (overwritten).
- contours saved under ../_temp/output/ex6\contours_ex6.csv (overwritten).


**Note:** the countour csv contains BOTH the outer (unseparated="parent") and inner (separated="child") contours. Be sure to select the desired rows using the `"order"` column when analyzing.